In [413]:
#outputs all variables on their own lines in the OUT console

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [414]:
from bs4 import BeautifulSoup as bs
import os
import urllib3 
import subprocess
import requests

In [578]:
#Pre-soup methods (URL request -> page contents -> text)


def getPageText(selector):
    try: 
        sourcepages.get(selector)
        response=requests.get(sourcepages.get(selector))
        PAGE=response.text
        print (len(PAGE))
        return PAGE, len(PAGE)
    except ValueError:
        #print (len(PAGE))   #MissingSchema exception
        print ("getPageText: Not an appropriate page identifier: check sourcepages dict")
        return 0


def getPageTextTest(selector):
    try:
        assert getPageText(selector) != 0
    except AssertionError:
        print("getPageTextTest: Entered an invalid page identifier: ", selector)
    
#a test suite for the above
def testPageGetting(selector):
    getPageTextTest(selector)
    #getPageText(selector)  #test that this fails properly - and it does

In [616]:
class specialSoup(bs):
    tagtarget="li"
    
    def findAllTargets(self, tagtarget):
        """gets litargs array"""
        targettags=self.find_all(tagtarget)
        #print("Happy class method findAllTargets")
        return targettags

In [423]:
def findPOIrecords(targetsarray):
    """extract POI records; gets recsarray from litargs"""
    recsarray=[]
    for tag in targetsarray:
        if tag.attrs["class"][0]=="c-media__list":
            recsarray.append(tag)
    return recsarray


In [473]:
sourcepages={
"tour" : "https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/?cost=min-max",
"eat" : "https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max",
"shop" : "https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max",
}

selector = "shop"
shoppage = getPageText(selector)
shoppage = shoppage[0]  #bc. tuple is returned with PAGE text (string), and size

tourpage=getPageText("tour")
tourpage=tourpage[0]

eatpage=getPageText("eat")
eatpage=eatpage[0]

SOUP = bs(shoppage, "html5lib")
toursoup = bs(tourpage, "html5lib")
eatsoup = bs(eatpage, "html5lib")

#eatsoup
#toursoup

93738
103002
105972


In [510]:
##next step: get the multiple sourcepages for shop, tour, eat and put in arrays,
#run above app by making selector from each of these

shoppages=[]
tourpages=[]
eatpages=[]

shoppages.append(sourcepages.get("shop"))
tourpages.append(sourcepages.get("tour"))
eatpages.append(sourcepages.get("eat"))

shoppages
tourpages
eatpages

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max']

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/?cost=min-max']

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max']

In [511]:
target[0].attrs
target[0]["href"]  #next page href
shoppages.append(target[0]["href"])
shoppages

{'class': ['c-pagenarion__navi'],
 'href': 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/p-2/?cost=min-max'}

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/p-2/?cost=min-max'

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max',
 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/p-2/?cost=min-max']

In [512]:
#put the above two cells into a method to return the next page
#DEPRECATED
def getNextPage(soup):
    atags=soup.find_all("a")
    target=[]
    nextpage=[]
    for tag in atags:
        try:
            if ("c-pagenarion__navi") in tag["class"]:
                target.append(tag)
        except KeyError:
            pass
    
    nextpage=target[0]["href"]
    return nextpage

In [666]:
#put the above two cells into a method to return the next page
def getNextPage2(soup):
    atags=soup.find_all("a")
    target=[]
    nextpage=[]
    for tag in atags:
        try:
            if ("c-pagenarion__navi") in tag["class"]:
                if tag.span["class"][1] == "--mono-base-arrow":
                    target.append(tag)
        except KeyError:
            pass
    
    nextpage=target[0]["href"]
    return nextpage

def getNextPagefromURL(url):
    urltext=(getPage(url))[0]
    soup=specialSoup(urltext, "html5lib")
    
    atags=soup.find_all("a")
    target=[]
    nextpage=[]
    for tag in atags:
        try:
            if ("c-pagenarion__navi") in tag["class"]:
                if tag.span["class"][1] == "--mono-base-arrow":
                    target.append(tag)
        except KeyError:
            pass
    
    if len(target)==0:
        print("No next page")
        nextpage=""
    else:
        nextpage=target[0]["href"]
        
    return nextpage

In [540]:
#getting second page for each
shoppages
tourpages
eatpages

if getNextPage(SOUP) in shoppages:
    print("Already there")
else:
    shoppages.append(getNextPage(SOUP))
    
if getNextPage(toursoup) in tourpages:
    print("Already there")
else:
    tourpages.append(getNextPage(toursoup))

if getNextPage(eatsoup) in eatpages:
    print("Already there")
else:
    eatpages.append(getNextPage(eatsoup))

shoppages
tourpages
eatpages

sourcepages["shop2"]=shoppages[1]
sourcepages["tour2"]=tourpages[1]
sourcepages["eat2"]=eatpages[1]

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max',
 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/p-2/?cost=min-max']

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/?cost=min-max',
 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/p-2/?cost=min-max']

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max',
 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/p-2/?cost=min-max']

Already there
Already there
Already there


['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max',
 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/p-2/?cost=min-max']

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/?cost=min-max',
 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/p-2/?cost=min-max']

['https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max',
 'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/p-2/?cost=min-max']

In [428]:
mySpecialSoup=specialSoup(shoppage, "html5lib")
tagtarg="li"
lis=mySpecialSoup.findAllTargets(tagtarg)
recs=findPOIrecords(lis)
print(len(recs))

#mySpecialSoup.find_all(mySpecialSoup.tagtarget)

Happy class method findAllTargets
10


In [538]:
#shop page 1
orderedRecs=orderRecords(recs)
for i in orderedRecs: print(i)
    
print(len(orderedRecs))

{'locn': '渋谷', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0003254/', 'id': 'lj0003254', 'name': '東急百貨店東横店'}
{'locn': '池袋', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0002989/', 'id': 'lj0002989', 'name': '東武百貨店 池袋店'}
{'locn': '新宿', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shinjuku/spot-lj0003209/', 'id': 'lj0003209', 'name': '小田急百貨店 新宿店'}
{'locn': '東京駅', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-tokyo_train_station/spot-lj0005375/', 'id': 'lj0005375', 'name': '日本橋高島屋'}
{'locn': '渋谷', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0005642/', 'id': 'lj0005642', 'name': '西武渋谷店'}
{'locn': '池袋', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0005640/', 'id': 'lj0005640', 'name': '西武池袋本店'}
{'locn': '渋谷', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0003256/', 'id': 'lj0003256', 'name': '東急百貨店本店'}
{'locn': '', 'type': '', 'hr

In [539]:
#opening the next page: can do 1 of 2 ways - 
#via "next" button on each page using getNextPage above
#or, just adding URLs until not able to get any new ones
shoppages[0]

spage1=getPageText("shop")
spage2 = getPageText("shop2")

#sourcepages
#spage2 = spage2[0]  #bc. tuple is returned with PAGE text (string), and size

#shopsoup2 = bs(spage2[0], "html5lib")
shopsoup2 = specialSoup(spage2[0], "html5lib")
tagtarg="li"
ss2li=shopsoup2.findAllTargets(tagtarg)
print(len(ss2li))
ss2recs=findPOIrecords(ss2li)
print(len(ss2recs))

ss2orderedrecs=orderRecords(ss2recs)
for i in ss2orderedrecs: print(i)
    
#shoppage2
print(len(ss2orderedrecs))

shoprecs=orderedRecs
for i in ss2orderedrecs:
    if i not in shoprecs:
        shoprecs.append(i)
    else:
        print(i, "Already in the shoprecs")

#our final shop list
print(shoprecs)
print(len(shoprecs))

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max'

93738
53566
Happy class method findAllTargets
29
6
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-tokyo_train_station/spot-lj0000682/', 'id': 'lj0000682', 'name': '大丸東京店'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0002143/', 'id': 'lj0002143', 'name': '西武池袋本店'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-tokyo_train_station/spot-lj0002122/', 'id': 'lj0002122', 'name': '日本橋三越本店'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-ueno/spot-lj0003041/', 'id': 'lj0003041', 'name': '松坂屋\u3000上野店'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-ginza/spot-lj0000683/', 'id': 'lj0000683', 'name': '松屋銀座'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-asakusa/spot-lj0003043/', 'id': 'lj0003043', 'name': '松屋浅草'}
6
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0002143/', 'id': 'lj0002143', 'name': '西武池袋本店'} Already in the shoprecs
[{'locn': '渋谷', 'type':

In [ ]:
tourpages
tourpage2=getPageText("tour2")
tourpage2=tourpage2[0]

tour2soup=specialSoup(tourpage2, "html5lib")
#tour2soup

getNextPage2(toursoup)
getNextPage2(tour2soup)

In [598]:
def getPage(url):
    try:
        response=requests.get(url)
        PAGE=response.text
        #print (len(PAGE))
        return PAGE, len(PAGE)
    except ValueError:
        return "Nope Nope Nope"
        
tpage="https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/p-72/?cost=min-max"
epage="https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/p-3/?cost=min-max"
teat=sourcepages.get("eat")
print(teat)

#below still works, just loops over to the next page
#oopage="https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/p-75/?cost=min-max"

tpagetext=getPage(tpage)
epagetext=getPage(epage)
#print(tpagetext[1])
tpagetext=tpagetext[0]
epagetext=epagetext[0]

tpagesoup=specialSoup(tpagetext, "html5lib")
epagesoup=specialSoup(epagetext, "html5lib")
tagtarg="li"
lis=tpagesoup.findAllTargets(tagtarg)
elis=epagesoup.findAllTargets(tagtarg)

recs=findPOIrecords(lis)
print(len(recs), " records on page\n")

erecs=findPOIrecords(elis)
print(len(erecs), "eat records on page\n")

orderedRecs=orderRecords(recs)
for i in orderedRecs: print(i)

eorderedRecs=orderRecords(erecs)
for i in eorderedRecs: print(i)

https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max
Happy class method findAllTargets
Happy class method findAllTargets
7  records on page

10 eat records on page

{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-tsukiji/spot-lj0003568/', 'id': 'lj0003568', 'name': '波除稲荷神社'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0002566/', 'id': 'lj0002566', 'name': '宮下公園'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-kagurazaka/spot-lj0003610/', 'id': 'lj0003610', 'name': '神楽坂通り'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-tokyo_train_station/spot-lj0005612/', 'id': 'lj0005612', 'name': '日本橋とやま館 観光交流サロン'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-ningyocho_monzen-nakacho_kasai/spot-lj0000014/', 'id': 'lj0000014', 'name': 'T-CAT外国人観光案内所'}
{'locn': '', 'type': '', 'href': 'https://livejapan.com/ja/in-itabashi_nerima/spot-lj0002406/', 'id': 'lj0002406', 'name': '大泉ア

In [ ]:
def getPOIsfromURL(url):
    urltext=(getPage(url))[0]
    pagesoup=specialSoup(urltext, "html5lib")
    tagtarget="li"
    litags=pagesoup.findAllTargets(tagtarget)
    
    recs=findPOIrecords(litags)
    POIRecs=orderRecords(recs)
    
    return POIRecs

def orderRecords(recsarray):
    """get your item records in a nice orderly list of dictionaries"""
    recordlist=[]
    for record in recsarray:
        recordcontents={}
        innertags=[]
        recordcontents["id"]=record["data-spotid"]
        
        for tag in record.descendants:
            innertags.append(tag)
        
        titles=(innertags[12].text).split()
        #print(titles)
                
        recordcontents["href"]=innertags[5].attrs["href"]
        recordcontents["name"]=innertags[8].attrs["alt"]
        try:
            recordcontents["locn"]=titles[1]
            recordcontents["type"]=titles[2]
        except IndexError:
            recordcontents["locn"]=innertags[27].title().strip()
            recordcontents["type"]=innertags[29].title().strip()
    
        recordlist.append(recordcontents)
    return recordlist   #if this is off by one indent, will overwrite each time and return only the last record value    


r=getPOIsfromURL(teat)
for i in r: print(i)

In [ ]:
#Next, to get ALL the poi pages
#two possibilities: just call method getPagesfromURL and add to array
#or, get page urls first and store them in array
tt=sourcepages.get("tour")
st=sourcepages.get("shop")

epage
getNextPagefromURL(epage)

#if no next page, print 0
tpage
len(getNextPagefromURL(tpage))

teat
getNextPagefromURL(teat)

In [ ]:
allshopresults=[]
alleatresults=[]
alltourresults=[]

nextexists=1
result=st
while len(result) > 0:
    allshopresults.append(result)
    result=getNextPagefromURL(result)

# print(allshopresults)
# print(alleatresults)
# print(alltourresults)

def getAllPages(url):
    result=url
    resultsindex=[]
    resultsindex.append(url)
    
    while (len(getNextPagefromURL(url))) > 0:
        url=getNextPagefromURL(url)
        resultsindex.append(url)
    
# url is not getting updated here;  so always reading the initial url value  
#     while len(getNextPagefromURL(url))>0:
#         resultsindex.append(result)
#         result=getNextPagefromURL(result)
        
    return resultsindex


print(getAllPages(st))
print(getAllPages(tt))
        
    
#getNextPagefromURL(tt)
#len(getNextPagefromURL(getNextPagefromURL(getNextPagefromURL(st))))

In [701]:
st=sourcepages.get("shop")
tt=sourcepages.get("tour")
et=sourcepages.get("eat")

st
tt
et

#these are the pages we will look through
shop_all=(getAllPages(st))
tt_all=(getAllPages(tt))
et_all=(getAllPages(et))

print(len(shop_all), len(tt_all), len(et_all))

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cm0301/?cost=min-max'

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/?cost=min-max'

'https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl02/?cost=min-max'

No next page
No next page
No next page
2 72 169


In [ ]:
shopPOIlist=[]
eatPOIlist=[]
tourPOIlist=[]

for surl in shop_all:
    for item in getPOIsfromURL(surl):
        if item not in shopPOIlist:
            shopPOIlist.append(item)
            
    
for turl in tt_all:
    for item in getPOIsfromURL(turl):
        if item not in tourPOIlist:
            tourPOIlist.append(item)
    
for eurl in et_all:
    for item in getPOIsfromURL(eurl):
        if item not in eatPOIlist:
            eatPOIlist.append(item)

#our final output arrays
print(len(shopPOIlist), len(eatPOIlist), len(tourPOIlist))

In [727]:
r=["hi", "ue", "hin", "ooo"]
n=" ".join(r[2:])
print(n)

hin ooo


In [724]:
#our final output array items
for i in shopPOIlist:
    print(i)

{'locn': '池袋', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0002989/', 'id': 'lj0002989', 'name': '東武百貨店 池袋店'}
{'locn': '渋谷', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0003254/', 'id': 'lj0003254', 'name': '東急百貨店東横店'}
{'locn': '新宿', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shinjuku/spot-lj0003209/', 'id': 'lj0003209', 'name': '小田急百貨店 新宿店'}
{'locn': '東京駅', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-tokyo_train_station/spot-lj0005375/', 'id': 'lj0005375', 'name': '日本橋高島屋'}
{'locn': '渋谷', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0005642/', 'id': 'lj0005642', 'name': '西武渋谷店'}
{'locn': '池袋', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-ikebukuro/spot-lj0005640/', 'id': 'lj0005640', 'name': '西武池袋本店'}
{'locn': '渋谷', 'type': '百貨店・デパート', 'href': 'https://livejapan.com/ja/in-shibuya/spot-lj0003256/', 'id': 'lj0003256', 'name': '東急百貨店本店'}
{'locn': '新宿', 'type': '百貨店・

In [707]:
shops= [item for sublist in shopPOIlist for item in sublist]
print(len(shops))    

16


In [697]:
atags=tour2soup.find_all("a")
len(atags)
target=[]
for n in atags:
    try: 
        #print(type(n["class"]))
        #print ("c-pagenarion__navi" in n["class"])
        if ("c-pagenarion__navi") in n["class"]:     #apparently == True, == False does not work
            #print(n)
            print(n.span)
            print(n.span.text)
            print(n.span["class"][1])
            if n.span["class"][1] == "--mono-base-arrow":
                print("THISSSSS")
                target.append(n)
    except KeyError:
        pass

print(target)

55

<span class="c-ico --mono-base-arrow-l"></span>

--mono-base-arrow-l
<span class="c-ico --mono-base-arrow"></span>

--mono-base-arrow
THISSSSS
[<a class="c-pagenarion__navi" href="https://livejapan.com/ja/in-pref-tokyo/spot-list/area-al1301/cate-cl01/p-3/?cost=min-max"><span class="c-ico --mono-base-arrow"></span></a>]


In [ ]:
#some tests of recsarray  
#print(len(recsarray))  #10
# print (recsarray[0])  #first POI record
# type(recsarray[0])  #<class 'bs4.element.Tag'>

#dictionary functions test code
def dictTest():
    tag0info={}
    tag0info["hi"]="b"
    tag0info["bn"]="ue"
    tag0info["hrr"]=None
    tag0info["hrr"]="he"
    print(tag0info)
    print(tag0info.keys())
    print(tag0info.values())

In [408]:
#Test codes
sourcepages.get("shop")
testPageGetting("uu")
dictTest()

getPageText: Not an appropriate page identifier: check sourcepages dict
getPageTextTest: Entered an invalid page identifier:  uu
{'bn': 'ue', 'hi': 'b', 'hrr': 'he'}
dict_keys(['bn', 'hi', 'hrr'])
dict_values(['ue', 'b', 'he'])


In [409]:
#testtag=recsarray[0]

In [431]:
# orderedRecs=orderRecords(recsarray)
# print(len(recsarray))

# for i in orderedRecs: print(i)
    

In [432]:
# #get id
# tag0={}
# testtag["data-spotid"]
# tag0["id"]=testtag["data-spotid"]
# print(tag0)

In [433]:
# #get href
# count=0
# tarray=[]
# for a in testtag.descendants:   #also tags
#     tarray.append(a)
#     count+=1

# print(type(tarray[3]))

# tag0["href"]=tarray[5].attrs["href"]
# tag0["name"]=tarray[8].attrs["alt"]   #or tarray[24]
# tag0["locn"]=tarray[27].title().strip()
# tag0["type"]=tarray[29].title().strip()
# print(tag0)
# print(tag0.keys())
# print(tag0.values())
# print(tag0["name"])

# print(type(tarray[27]))
# locn=tarray[27]
# typename=tarray[29]


In [434]:
# litagsarray=[]
# for tag in litags:
#     litagsarray.append(tag)
#     #print(tag.attrs)
    
# print(type(litagsarray[0]))    #each element is still a bs4.element.Tag
# for tag in litagsarray:
# #     print(tag.attrs)
# #     print(tag['class'])
#     if tag['class'] [0]== 'c-media__list':
#         print(tag.attrs)
#         print(tag)
#         print("THIS")

In [435]:
# #prints a single location tag

# tags=SOUP.body
# #print(tags)
# #lrtags=SOUP.find_all(class ="c-media__list")
# print(len(litags))
# #print(len(lrtags))

# #ltags=SOUP.find_all("id")

# #len(ltags)
# rectags=[]
# spottags=[]
# for i in litags:
#     if str(i).find("c-media__list") > -1:
# #     if str(i).find("m_recommend__img") > -1:   #if not found, gives index of -1, if found, gives index
#         spottags.append(str(i))
# #         print(i)
#     else:
#         pass
    
# #len(rectags)
# # rectags[8]   #this is the row of recommended images
# #len(atags)
# print(len(spottags))
# print(spottags[9])
# print(spottags[9].find("spot-lj"))

In [347]:
# def orderRecords(recsarray):
#     recordlist=[]
#     for record in recsarray:
#         recordcontents={}
#         innertags=[]
#         recordcontents["id"]=record["data-spotid"]
        
#         for tag in record.descendants:
#             innertags.append(tag)
            
#         recordcontents["href"]=innertags[5].attrs["href"]
#         recordcontents["name"]=innertags[8].attrs["alt"]
#         recordcontents["locn"]=innertags[27].title().strip()
#         recordcontents["type"]=innertags[29].title().strip()
    
#         recordlist.append(recordcontents)
#     return recordlist   #if this is off by one indent, will overwrite each time and return only the last record value
    

In [346]:
# recordlist=[]
# for record in recsarray:
#     recordcontents={}
#     inner=[]
#     recordcontents["id"]=record["data-spotid"]
    
#     for tag in record.descendants:
#         inner.append(tag)
    
#     recordcontents["href"]=inner[5].attrs["href"]
#     recordcontents["name"]=inner[8].attrs["alt"]
#     recordcontents["locn"]=inner[27].title().strip()
#     recordcontents["type"]=inner[29].title().strip()
    
#     recordlist.append(recordcontents)
    
# #print(recordlist)
# for a in recordlist:
#     print(a)
    

In [405]:
#gets rid of whitespace in name tags
# typename
# type(typename)
# ?typename.title
# print(typename.title().strip())     #shibuya  (without spaces)
# print(locn.title().strip())


In [395]:
#len(SOUP) #2
#len(SOUP.prettify()) #86650
#print(SOUP.prettify())
# print(type(SOUP))      #<class 'bs4.BeautifulSoup'>

#some SOUP tests:
#SOUP.a  #prints the first "a" tag
divtags=SOUP.find_all("div")
#len(divtags)  #194 divtags in total
#divtags[120]

atags=SOUP.find_all("a")
#SOUP.find("c-media__area")    #will not find this, as this is a class name, not a tag type

In [420]:
litags=SOUP.find_all("li")
#print(type(litags[0]))      #litags is a ResultSet, litags[0] is a bs4.element.Tag object
print(type(litags[0]))
testtag=litags[0]
testtag.attrs
#print(testtag.contents)
#print(testtag)

print(testtag.id)
testtag.has_attr("class")
testtag.attrs["class"]

recsarray=[]
for tag in litags:
    print (tag.attrs["class"])
    if tag.attrs["class"][0] =="c-media__list": 
        print ("YEA")
        recsarray.append(tag)

<class 'bs4.element.Tag'>


{'class': ['header__navi__list']}

None


True

['header__navi__list']

['header__navi__list']
['header__navi__list']
['header__navi__list']
['header__navi__list']
['header__navi__list']
['header__navi__list']
['recommend__list']
['recommend__list']
['recommend__list']
['recommend__list']
['recommend__list']
['recommend__list']
['recommend__list']
['recommend__list']
['recommend__list']
['c-media__list', '--column']
YEA
['c-group__item', 'tooltip']
['c-media__list', '--column']
YEA
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-media__list', '--column']
YEA
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-media__list', '--column']
YEA
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-media__list', '--column']
YEA
['c-group__item', 'tooltip']
['c-group__item', 'tooltip']
['c-group__item', '